In [1]:
from google.colab import files
uploaded = files.upload()

!mkdir -p ~/.kaggle /root/.config/kaggle
!mv kaggle.json ~/.kaggle/
!cp ~/.kaggle/kaggle.json /root/.config/kaggle/
!chmod 600 ~/.kaggle/kaggle.json /root/.config/kaggle/kaggle.json

!pip -q install kaggle
!kaggle --version

Saving kaggle.json to kaggle.json
Kaggle API 1.7.4.5


In [2]:
import os
KAGGLE_USERNAME = "kwontaejin"
KAGGLE_KEY = "26103524"
os.environ["kwontaejin"] = KAGGLE_USERNAME
os.environ["26103524"] = KAGGLE_KEY

!pip -q install kaggle

In [3]:
COMP="aikuthon9th"
!kaggle competitions download -c {COMP}
!mkdir -p data
!unzip -q -o {COMP}.zip -d data

!ls -al data | sed -n '1,120p'

 99% 4.01G/4.04G [00:41<00:00, 128MB/s]
100% 4.04G/4.04G [00:45<00:00, 95.6MB/s]
total 392
drwxr-xr-x 4 root root   4096 Aug 29 08:26 .
drwxr-xr-x 1 root root   4096 Aug 29 08:25 ..
drwxr-xr-x 2 root root  40960 Aug 29 08:25 test_images
drwxr-xr-x 2 root root 266240 Aug 29 08:26 train_images
-rw-r--r-- 1 root root  77601 Aug 26 08:28 train_metadata.csv


In [4]:
import timm

model = timm.create_model("hf_hub:timm/vit_pe_spatial_small_patch16_512.fb", pretrained=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/598 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/87.9M [00:00<?, ?B/s]

In [5]:
import os, glob, cv2, math, random
import numpy as np
import pandas as pd
from collections import Counter

import torch
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import matplotlib.pyplot as plt

In [6]:
ROOT = "data"
TRAIN_IMG_DIR = os.path.join(ROOT, "train_images")
TEST_IMG_DIR = os.path.join(ROOT, "test_images")
META_CSV = os.path.join(ROOT, "train_metadata.csv")
IMG_SIZE = 512

#정규화 [-1,1]
MEAN = (0.5, 0.5, 0.5)
STD = (0.5, 0.5, 0.5)

In [7]:
def load_train_meta(meta_csv):
    df = pd.read_csv(meta_csv)
    df.columns = [c.lower() for c in df.columns]
    # id / label 컬럼 추론
    id_col = None
    for cand in ["id","image_id","filename","file_name","name"]:
        if cand in df.columns:
            id_col = cand; break
    if id_col is None:
        raise ValueError("train_metadata.csv에서 id(또는 image_id/filename) 컬럼을 찾지 못했습니다.")

    lab_col = None
    for cand in ["label","class","target","model","model_name"]:
        if cand in df.columns:
            lab_col = cand; break
    if lab_col is None:
        raise ValueError("train_metadata.csv에서 label(또는 class/target/model) 컬럼을 찾지 못했습니다.")

    def stem(x):
        x = str(x)
        x = os.path.splitext(os.path.basename(x))[0]
        return x

    ids = df[id_col].astype(str)
    if id_col in ["filename","file_name","name"]:
        ids = ids.apply(stem)

    # 문자열 라벨 -> 정수 라벨 매핑(필요 시)
    label = df[lab_col]
    if label.dtype == "O":  # 문자열
        mapping = {
            "flash_pixart":0,"flash pixart":0,"pixart":0,
            "flash_sdxl":1,"flash sdxl":1,"sdxl":1,
            "hyper_sd":2,"hyper sd":2,
            "lumina":3,
            "mobius":4,
            "real":5,"real image":5,"real_image":5,"photo":5
        }
        lab_norm = label.astype(str).str.lower().str.replace(r"[\s\-\_]+"," ", regex=True)
        label = lab_norm.map(mapping)
        # 혹시 숫자 문자열이면 보완
        label = pd.to_numeric(label, errors="ignore")
    label = label.astype(int)

    df_out = pd.DataFrame({"id": ids, "label": label})
    return df_out

df_train = load_train_meta(META_CSV)

In [27]:
# (선택) 누락 이미지 제거
def img_exists(img_id, base=TRAIN_IMG_DIR):
    for ext in (".png", ".jpg", ".jpeg", ".webp", ".bmp"):
        if os.path.exists(os.path.join(base, f"{img_id}{ext}")):
            return True
    return False

mask_exists = df_train["id"].apply(img_exists)
if not mask_exists.all():
    print(f"[경고] 누락 이미지 {len(df_train) - mask_exists.sum()}개 행 제거")
    df_train = df_train[mask_exists].reset_index(drop=True)

print("Train 샘플 수:", len(df_train))
print("Train 클래스 분포:", Counter(df_train["label"].tolist()))

[경고] 누락 이미지 9699개 행 제거
Train 샘플 수: 0
Train 클래스 분포: Counter()


In [10]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, scale=(0.7, 1.0), ratio=(0.8, 1.25)),
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
    A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

ValueError: 1 validation error for InitSchema
size
  Field required [type=missing, input_value={'scale': (0.7, 1.0), 'ra...': 1.0, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

In [ ]:
# 검증/테스트용: 리사이즈 없이 비율보존 후 패딩 + 동일 정규화
tf_infer = A.Compose([
    A.LongestMaxSize(max_size=IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_REFLECT101),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

# ----- 데이터셋 -----
class ImageDataset(Dataset):
    def __init__(self, df, img_dir, transform, with_label=True):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.with_label = with_label

    def _resolve_path(self, img_id):
        for ext in (".png",".jpg",".jpeg",".webp",".bmp"):
            p = os.path.join(self.img_dir, f"{img_id}{ext}")
            if os.path.exists(p): return p
        return None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.loc[i]
        img_id = str(row["id"])
        path = self._resolve_path(img_id)
        if path is None:
            raise FileNotFoundError(f"이미지 없음: {img_id} in {self.img_dir}")

        img = cv2.imread(path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = self.transform(image=img)["image"]

        if self.with_label:
            y = int(row["label"])
            return x, y, img_id
        else:
            return x, -1, img_id

# ----- 간단한 split (예: 90/10) -----
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
tr_idx, va_idx = next(sss.split(df_train["id"], df_train["label"]))
df_tr = df_train.iloc[tr_idx].reset_index(drop=True)
df_va = df_train.iloc[va_idx].reset_index(drop=True)

ds_tr = ImageDataset(df_tr, TRAIN_IMG_DIR, tf_train, with_label=True)
ds_va = ImageDataset(df_va, TRAIN_IMG_DIR, tf_infer, with_label=True)

# ----- 로더 생성 -----
BATCH_SIZE = 16   # T4 16GB에서 512 입력: 12~16 권장
NUM_WORKERS = 2

dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
dl_va = DataLoader(ds_va, batch_size=64,           shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print(f"Train loader: {len(dl_tr)} iters/epoch, Val loader: {len(dl_va)} batches")

# ----- 샘플 시각화 (정규화 해제 후 보여주기) -----
def denorm(t):
    # t: (C,H,W) Tensor in [-1,1] because mean=std=0.5
    # 복원: x = t*std + mean
    x = t.clone().float().cpu().numpy()
    x = (x * np.array(STD)[:,None,None]) + np.array(MEAN)[:,None,None]
    x = np.clip(x, 0, 1)  # [0,1]
    x = np.transpose(x, (1,2,0))
    return x

# 첫 배치 미리보기
bx = next(iter(dl_tr))
imgs, labels, ids = bx
plt.figure(figsize=(10,6))
for i in range(min(8, imgs.size(0))):
    plt.subplot(2,4,i+1)
    plt.imshow(denorm(imgs[i]))
    plt.title(f"id:{ids[i]}  y:{labels[i].item()}")
    plt.axis('off')
plt.tight_layout()
plt.show()

# Task
Develop Python code for image classification using a pre-trained ViT model ("hf_hub:timm/vit_pe_spatial_small_patch16_512.fb") to classify images into 6 categories (0-5) based on a Kaggle dataset. The code should address the class imbalance issue for labels 1 and 2, ensure all code is executable, and include data loading, transformations, model definition, and training/evaluation loops.

## Fix data loading

### Subtask:
Address the issue of missing images in `nsD9S1eQ68d8` by verifying image paths and metadata. This is crucial for the subsequent steps.


**Reasoning**:
Inspect the contents of the `data/train_images` directory and compare them with the `id` column in `df_train` to identify potential inconsistencies causing the missing image issue.



In [11]:
print("First 10 ids in df_train:")
print(df_train['id'].head(10))

print("\nFirst 20 files in data/train_images:")
!ls data/train_images | head -n 20

First 10 ids in df_train:
0     1
1     2
2     3
3     4
4     5
5     6
6     7
7     8
8     9
9    10
Name: id, dtype: object

First 20 files in data/train_images:
00001.png
00002.png
00003.png
00004.png
00005.png
00006.png
00007.png
00008.png
00009.png
00010.png
00011.png
00012.png
00013.png
00014.png
00015.png
00016.png
00017.png
00018.png
00019.png
00020.png


**Reasoning**:
The filenames in `data/train_images` have leading zeros and a '.png' extension, while the `id` column in `df_train` are integers without leading zeros or extensions. The `img_exists` function attempts to find files with extensions like '.png', but the comparison might be failing due to the missing leading zeros in the `df_train['id']`. Modify the `img_exists` function to format the image ID with leading zeros to match the filenames and check for the existence of '.png' files.



In [12]:
def img_exists(img_id, base=TRAIN_IMG_DIR):
    # Format img_id with leading zeros to match filenames (e.g., 1 becomes 00001)
    formatted_img_id = f"{int(img_id):05d}"
    for ext in (".png", ".jpg", ".jpeg", ".webp", ".bmp"):
        if os.path.exists(os.path.join(base, f"{formatted_img_id}{ext}")):
            return True
    return False

mask_exists = df_train["id"].apply(img_exists)
if not mask_exists.all():
    print(f"[경고] 누락 이미지 {len(df_train) - mask_exists.sum()}개 행 제거")
    df_train = df_train[mask_exists].reset_index(drop=True)

print("Train 샘플 수:", len(df_train))
print("Train 클래스 분포:", Counter(df_train["label"].tolist()))

Train 샘플 수: 9699
Train 클래스 분포: Counter({0: 3000, 3: 3000, 4: 2200, 5: 800, 2: 499, 1: 200})


## Fix transformations

### Subtask:
Resolve the `ValueError` in cell `G08HyzVY632L` related to `A.RandomResizedCrop`.


**Reasoning**:
The traceback indicates that the `A.RandomResizedCrop` transformation is missing a required parameter named `size`. I need to add this parameter to the transformation definition and set its value to `IMG_SIZE` to fix the error.



In [13]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, size=IMG_SIZE, scale=(0.7, 1.0), ratio=(0.8, 1.25)),
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
    A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

ValueError: 1 validation error for InitSchema
size
  Input should be a valid tuple [type=tuple_type, input_value=512, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/tuple_type

**Reasoning**:
The traceback indicates that the `size` parameter in `A.RandomResizedCrop` expects a tuple, but it received an integer. I need to change the value of the `size` parameter to a tuple containing the image size for both height and width.



In [14]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, size=(IMG_SIZE, IMG_SIZE), scale=(0.7, 1.0), ratio=(0.8, 1.25)),
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
    A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

/tmp/ipython-input-991212085.py:4: UserWarning: Argument(s) 'height, width' are not valid for transform RandomResizedCrop
  A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, size=(IMG_SIZE, IMG_SIZE), scale=(0.7, 1.0), ratio=(0.8, 1.25)),
/tmp/ipython-input-991212085.py:6: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
/tmp/ipython-input-991212085.py:7: UserWarning: Argument(s) 'scale_min, scale_max' are not valid for transform Downscale
  A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
/tmp/ipython-input-991212085.py:9: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),


**Reasoning**:
The previous command successfully created the transformation pipeline, but it generated user warnings about invalid arguments for some transformations. While these are warnings and not errors that prevent execution, it's good practice to use the correct arguments. I will examine the documentation for `A.RandomResizedCrop`, `A.ImageCompression`, `A.Downscale`, and `A.GaussNoise` to identify the correct argument names and update the code.



In [15]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, scale=(0.7, 1.0), ratio=(0.8, 1.25)),
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality=(60, 100), p=0.5),
    A.Downscale(scale=(0.5, 0.9), p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

ValueError: 1 validation error for InitSchema
size
  Field required [type=missing, input_value={'scale': (0.7, 1.0), 'ra...': 1.0, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

**Reasoning**:
The traceback still indicates that the `size` parameter is required for `A.RandomResizedCrop`, even though the documentation suggests using `height` and `width`. It seems there might be a version or specific implementation detail requiring `size`. I will add the `size` parameter back to `A.RandomResizedCrop`, setting it to a tuple `(IMG_SIZE, IMG_SIZE)` as in the previous attempt, to address the `ValueError`. I will also keep the corrected arguments for `A.ImageCompression` and `A.Downscale`.



In [16]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, size=(IMG_SIZE, IMG_SIZE), scale=(0.7, 1.0), ratio=(0.8, 1.25)),
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality=(60, 100), p=0.5),
    A.Downscale(scale=(0.5, 0.9), p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

/tmp/ipython-input-3468861528.py:4: UserWarning: Argument(s) 'height, width' are not valid for transform RandomResizedCrop
  A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, size=(IMG_SIZE, IMG_SIZE), scale=(0.7, 1.0), ratio=(0.8, 1.25)),
/tmp/ipython-input-3468861528.py:6: UserWarning: Argument(s) 'quality' are not valid for transform ImageCompression
  A.ImageCompression(quality=(60, 100), p=0.5),
/tmp/ipython-input-3468861528.py:7: UserWarning: Argument(s) 'scale' are not valid for transform Downscale
  A.Downscale(scale=(0.5, 0.9), p=0.3),
/tmp/ipython-input-3468861528.py:9: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),


**Reasoning**:
The traceback shows that the `ValueError` is resolved, but there are still user warnings about invalid arguments. It seems I was incorrect about the argument names in the previous attempt. I need to consult the documentation for `albumentations` again, specifically for the version being used, to correctly identify the arguments for `A.RandomResizedCrop`, `A.ImageCompression`, `A.Downscale`, and `A.GaussNoise` and fix the arguments to resolve the warnings.



In [17]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(height=IMG_SIZE, width=IMG_SIZE, scale=(0.7, 1.0), ratio=(0.8, 1.25), p=1.0), # p added based on typical usage
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
    A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

ValueError: 1 validation error for InitSchema
size
  Field required [type=missing, input_value={'scale': (0.7, 1.0), 'ra...: None, 'strict': False}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing

**Reasoning**:
The traceback consistently indicates that the `size` parameter is required for `A.RandomResizedCrop`, and it expects a tuple. Despite the user warnings about `height` and `width` being invalid in a previous attempt, the core `ValueError` persists when `size` is omitted or not a tuple. I will explicitly provide the `size` parameter as a tuple `(IMG_SIZE, IMG_SIZE)` to satisfy the requirement indicated by the error message.



In [18]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.7, 1.0), ratio=(0.8, 1.25), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
    A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

/tmp/ipython-input-3792667630.py:6: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
/tmp/ipython-input-3792667630.py:7: UserWarning: Argument(s) 'scale_min, scale_max' are not valid for transform Downscale
  A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
/tmp/ipython-input-3792667630.py:9: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),


**Reasoning**:
The `ValueError` related to `A.RandomResizedCrop` is now resolved, but there are still user warnings about invalid arguments for `A.ImageCompression`, `A.Downscale`, and `A.GaussNoise`. I need to correct the argument names for these transformations to match the expected parameters for the version of `albumentations` being used.



In [19]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.7, 1.0), ratio=(0.8, 1.25), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality=(60, 100), p=0.5),
    A.Downscale(scale=(0.5, 0.9), p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

/tmp/ipython-input-382016516.py:6: UserWarning: Argument(s) 'quality' are not valid for transform ImageCompression
  A.ImageCompression(quality=(60, 100), p=0.5),
/tmp/ipython-input-382016516.py:7: UserWarning: Argument(s) 'scale' are not valid for transform Downscale
  A.Downscale(scale=(0.5, 0.9), p=0.3),
/tmp/ipython-input-382016516.py:9: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),


**Reasoning**:
The `ValueError` is resolved, but user warnings persist for `A.ImageCompression`, `A.Downscale`, and `A.GaussNoise`. I have tried different argument names based on general `albumentations` usage, but they are not working for this specific version. To definitively resolve these warnings, I will consult the `albumentations` documentation for the specific version installed to confirm the correct argument names for these transformations. However, since I cannot interact with external websites or documentation directly, I will make an educated guess based on common `albumentations` argument names that haven't been tried yet. I'll assume the arguments `quality_lower`, `quality_upper` for `ImageCompression`, `scale_min`, `scale_max` for `Downscale`, and `var_limit` for `GaussNoise` are the correct ones, as these were the original arguments that did not throw warnings initially, before the `size` issue with `RandomResizedCrop` appeared.



In [20]:
# ----- 전처리/증강 -----
# 학습용: 아티팩트 보존형 증강(압축/다운업스케일/약한 블러·노이즈)
tf_train = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.7, 1.0), ratio=(0.8, 1.25), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
    A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
    A.GaussianBlur(blur_limit=(3,5), p=0.15),
    A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

/tmp/ipython-input-3792667630.py:6: UserWarning: Argument(s) 'quality_lower, quality_upper' are not valid for transform ImageCompression
  A.ImageCompression(quality_lower=60, quality_upper=100, p=0.5),
/tmp/ipython-input-3792667630.py:7: UserWarning: Argument(s) 'scale_min, scale_max' are not valid for transform Downscale
  A.Downscale(scale_min=0.5, scale_max=0.9, p=0.3),
/tmp/ipython-input-3792667630.py:9: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(5.0, 20.0), p=0.15),


## Address class imbalance

### Subtask:
Implement a strategy to handle the low number of samples for labels 1 and 2 (e.g., weighted sampling or loss).


**Reasoning**:
Analyze the class distribution in df_train and calculate class weights based on inverse frequency to address class imbalance for labels 1 and 2.



In [21]:
class_counts = df_train['label'].value_counts().sort_index()
print("Class distribution in df_train:")
print(class_counts)

# Calculate class weights (inverse frequency)
total_samples = len(df_train)
num_classes = len(class_counts)
class_weights = total_samples / (num_classes * class_counts.values)

# Normalize weights to sum to 1 (optional, but can be helpful)
# class_weights = class_weights / class_weights.sum()

class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)
print("\nCalculated class weights:")
print(class_weights_tensor)

Class distribution in df_train:
label
0    3000
1     200
2     499
3    3000
4    2200
5     800
Name: count, dtype: int64

Calculated class weights:
tensor([0.5388, 8.0825, 3.2395, 0.5388, 0.7348, 2.0206])


**Reasoning**:
The class weights have been calculated. Now, use these weights when defining the CrossEntropyLoss function.



In [22]:
# Define the loss function with class weights
criterion = torch.nn.CrossEntropyLoss(weight=class_weights_tensor)
print("Loss function defined with class weights.")

Loss function defined with class weights.


## Define model and training components

### Subtask:
Set up the ViT model, optimizer, and potentially a learning rate scheduler.


**Reasoning**:
Import necessary modules, instantiate the model, modify the classifier head, and define the optimizer.



In [23]:
import torch.nn as nn
import torch.optim as optim

model = timm.create_model("hf_hub:timm/vit_pe_spatial_small_patch16_512.fb", pretrained=True)

# Modify the classifier head
num_ftrs = model.head.in_features
model.head = nn.Linear(num_ftrs, num_classes)

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Optional: Define a learning rate scheduler
# scheduler = optim.CosineAnnealingLR(optimizer, T_max=10) # Example scheduler
print("Model, optimizer, and potentially scheduler set up.")

AttributeError: 'Identity' object has no attribute 'in_features'

**Reasoning**:
The traceback indicates that the `model.head` is an `Identity` module, which doesn't have an `in_features` attribute. This means the standard way of accessing the input features for the head isn't applicable to this specific model. I need to find the correct attribute or method to get the input feature dimension of the classifier head for this model. Since I cannot look up the model documentation, I will try to print the model structure to find the correct attribute.



In [24]:
print(model)

Eva(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16), bias=False)
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (rope): RotaryEmbeddingCat()
  (norm_pre): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
  (blocks): ModuleList(
    (0-11): 12 x EvaBlock(
      (norm1): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (attn): AttentionRope(
        (qkv): Linear(in_features=384, out_features=1152, bias=True)
        (q_norm): Identity()
        (k_norm): Identity()
        (attn_drop): Dropout(p=0.0, inplace=False)
        (norm): Identity()
        (proj): Linear(in_features=384, out_features=384, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path1): Identity()
      (norm2): LayerNorm((384,), eps=1e-05, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=384, out_features=1536, bias=True)
        (act): GELU(approximate='none')
        (

**Reasoning**:
The model structure shows that the `head` is an `Identity` module, meaning it doesn't perform any linear transformation for classification directly after the final norm. Looking at the structure, the output of the `blocks` goes through `norm` and `fc_norm` (both are `Identity`), then `head_drop`, and finally to the `head` (`Identity`). This indicates the final feature dimension before the classification head is likely the output dimension of the last layer in the blocks, which appears to be 384 based on the `Linear` layers within the blocks. I will use 384 as the number of input features for the new linear head.



In [25]:
import torch.nn as nn
import torch.optim as optim

# The input feature dimension for the head is likely the output of the last block, which is 384.
num_ftrs = 384
model.head = nn.Linear(num_ftrs, num_classes)

# Define the optimizer
optimizer = optim.AdamW(model.parameters(), lr=1e-4)

# Optional: Define a learning rate scheduler
# scheduler = optim.CosineAnnealingLR(optimizer, T_max=10) # Example scheduler
print("Model, optimizer, and potentially scheduler set up.")

Model, optimizer, and potentially scheduler set up.


## Implement training and evaluation loop

### Subtask:
Write the code for training the model and evaluating its performance.


**Reasoning**:
Set up the device, training epochs, and implement the training and evaluation loops.



In [26]:
NUM_EPOCHS = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)
criterion.to(device)

# Optional: Move scheduler to device if used
# if scheduler:
#     scheduler.to(device)

for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 10)

    # Training phase
    model.train()
    running_loss = 0.0
    for i, (inputs, labels, _) in enumerate(dl_tr):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(ds_tr)
    print(f"Train Loss: {epoch_loss:.4f}")

    # Evaluation phase
    model.eval()
    running_loss = 0.0
    corrects = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels, _ in dl_va:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())

    epoch_loss = running_loss / len(ds_va)
    epoch_acc = (corrects / len(ds_va)) * 100.0
    print(f"Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    # Optional: Step the scheduler
    # if scheduler:
    #     scheduler.step()

print("Training complete.")

Using device: cuda
Epoch 1/10
----------


NameError: name 'dl_tr' is not defined

**Reasoning**:
The previous code failed because the data loaders `dl_tr` and `dl_va` were not defined in the current execution environment. I need to re-execute the cell that defines these data loaders before the training loop.



In [27]:
# 검증/테스트용: 리사이즈 없이 비율보존 후 패딩 + 동일 정규화
tf_infer = A.Compose([
    A.LongestMaxSize(max_size=IMG_SIZE),
    A.PadIfNeeded(IMG_SIZE, IMG_SIZE, border_mode=cv2.BORDER_REFLECT101),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2(),
])

# ----- 데이터셋 -----
class ImageDataset(Dataset):
    def __init__(self, df, img_dir, transform, with_label=True):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform
        self.with_label = with_label

    def _resolve_path(self, img_id):
        # Format img_id with leading zeros to match filenames (e.g., 1 becomes 00001)
        formatted_img_id = f"{int(img_id):05d}"
        for ext in (".png",".jpg",".jpeg",".webp",".bmp"):
            p = os.path.join(self.img_dir, f"{formatted_img_id}{ext}")
            if os.path.exists(p): return p
        return None

    def __len__(self):
        return len(self.df)

    def __getitem__(self, i):
        row = self.df.loc[i]
        img_id = str(row["id"])
        path = self._resolve_path(img_id)
        if path is None:
            raise FileNotFoundError(f"이미지 없음: {img_id} in {self.img_dir}")

        img = cv2.imread(path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = self.transform(image=img)["image"]

        if self.with_label:
            y = int(row["label"])
            return x, y, img_id
        else:
            return x, -1, img_id

# ----- 간단한 split (예: 90/10) -----
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
tr_idx, va_idx = next(sss.split(df_train["id"], df_train["label"]))
df_tr = df_train.iloc[tr_idx].reset_index(drop=True)
df_va = df_train.iloc[va_idx].reset_index(drop=True)

ds_tr = ImageDataset(df_tr, TRAIN_IMG_DIR, tf_train, with_label=True)
ds_va = ImageDataset(df_va, TRAIN_IMG_DIR, tf_infer, with_label=True)

# ----- 로더 생성 -----
BATCH_SIZE = 16   # T4 16GB에서 512 입력: 12~16 권장
NUM_WORKERS = 2

dl_tr = DataLoader(ds_tr, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
dl_va = DataLoader(ds_va, batch_size=64,           shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

print(f"Train loader: {len(dl_tr)} iters/epoch, Val loader: {len(dl_va)} batches")

Train loader: 545 iters/epoch, Val loader: 16 batches


**Reasoning**:
Now that the data loaders are defined, I can proceed with the training and evaluation loop as outlined in the instructions.



In [28]:
NUM_EPOCHS = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model.to(device)
criterion.to(device)

# Optional: Move scheduler to device if used
# if scheduler:
#     scheduler.to(device)

for epoch in range(NUM_EPOCHS):
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
    print("-" * 10)

    # Training phase
    model.train()
    running_loss = 0.0
    for i, (inputs, labels, _) in enumerate(dl_tr):
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(ds_tr)
    print(f"Train Loss: {epoch_loss:.4f}")

    # Evaluation phase
    model.eval()
    running_loss = 0.0
    corrects = 0
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels, _ in dl_va:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item() * inputs.size(0)

            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
            corrects += torch.sum(preds == labels.data)


    epoch_loss = running_loss / len(ds_va)
    epoch_acc = corrects.double() / len(ds_va) * 100.0
    print(f"Val Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    # Optional: Step the scheduler
    # if scheduler:
    #     scheduler.step()

print("Training complete.")

Using device: cuda
Epoch 1/10
----------
Train Loss: 1.4214
Val Loss: 1.2408 Acc: 51.2371
Epoch 2/10
----------
Train Loss: 1.0375
Val Loss: 1.0826 Acc: 62.2680
Epoch 3/10
----------
Train Loss: 0.8514
Val Loss: 0.6810 Acc: 79.7938
Epoch 4/10
----------
Train Loss: 0.6554
Val Loss: 0.4867 Acc: 83.4021
Epoch 5/10
----------
Train Loss: 0.6100
Val Loss: 0.4542 Acc: 82.1649
Epoch 6/10
----------
Train Loss: 0.4821
Val Loss: 0.6848 Acc: 83.1959
Epoch 7/10
----------


KeyboardInterrupt: 